In [67]:
import geopandas as gpd
import folium
import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join
from shapely.geometry import Polygon

import matplotlib.pyplot as plt


root = "C://Users//user//Desktop//Helmholtz//Tasks//Task 1//"


def get_all_basin_coords():
    basins_dir = "Basin_Boundaries//"
    mopex_dir = "MOPEX//"
    
    #basin_file_names = [f for f in listdir(join(root, basins_dir)) if isfile(join(root, basins_dir, f))]
    # below basins are present in mopex dataset
    basin_file_names = [f.replace('.txt', '.BDY') for f in listdir(join(root, mopex_dir)) if isfile(join(root, mopex_dir, f))]
    basin_file_names = basin_file_names[:-2]
    #basin_file_names = ["01048000.BDY", "01055500.BDY","01060000.BDY", "01064500.BDY", "01076500.BDY"]
        
    all_basin_geoms = []
    for file_name in basin_file_names[:]:
        lat_point_list = []
        lon_point_list = []
        df = pd.read_csv(join(root, basins_dir, file_name), delim_whitespace=True, header=None, skiprows=1)
        lat_point_list = df[1]
        lon_point_list = df[0]
    
        polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
        all_basin_geoms.append(polygon_geom)
         
    return all_basin_geoms, basin_file_names

 
def read_prism_hdr(hdr_path):
    """Read an ESRI BIL HDR file"""
    with open(hdr_path, 'r') as input_f:
        header_list = input_f.readlines()
    return dict(item.strip().split() for item in header_list)
 
def read_prism_bil(bil_path):
    """Read an array from ESRI BIL raster file"""
    hdr_dict = read_prism_hdr(bil_path.replace('.bil', '.hdr'))
    
    prism_array = np.fromfile(bil_path, dtype=np.float32)
    prism_array = prism_array.reshape(
        int(hdr_dict['NROWS']), int(hdr_dict['NCOLS']))
    prism_array[prism_array == float(hdr_dict['NODATA'])] = np.nan
    return prism_array

def get_ppt_data(year,month):
    prism_dir = "PRISM_ppt_stable_4kmM3_198101_202001_bil//"
    if(month<10):
        prism_file_path = "PRISM_ppt_stable_4kmM3_"+str(year)+"0"+str(month)+"_bil.bil"
    else:
        prism_file_path = "PRISM_ppt_stable_4kmM3_"+str(year)+str(month)+"_bil.bil"    
    ppt_data = read_prism_bil(join(root, prism_dir, prism_file_path))
    
    hdr_dict = read_prism_hdr(join(root, prism_dir, prism_file_path).replace('.bil', '.hdr'))
    
    hdr_dict["ULXMAP"] = float(hdr_dict["ULXMAP"])
    hdr_dict["ULYMAP"] = float(hdr_dict["ULYMAP"])
    hdr_dict['NROWS'] = int(hdr_dict['NROWS'])
    hdr_dict['NCOLS'] = int(hdr_dict['NCOLS'])
    hdr_dict['XDIM'] = float(hdr_dict['XDIM'])
    hdr_dict['YDIM'] = float(hdr_dict['YDIM'])
    
    p1 = (hdr_dict["ULXMAP"] - (hdr_dict['XDIM']/2), 
          hdr_dict["ULYMAP"] + (hdr_dict['XDIM']/2))

    p2 = (hdr_dict["ULXMAP"] + hdr_dict['NCOLS']*hdr_dict['XDIM'],
          hdr_dict["ULYMAP"] + (hdr_dict['XDIM']/2))

    p3 = (hdr_dict["ULXMAP"] + hdr_dict['NCOLS']*hdr_dict['XDIM'],
          hdr_dict["ULYMAP"] - hdr_dict['NROWS']*hdr_dict['YDIM'])

    p4 = (hdr_dict["ULXMAP"] - (hdr_dict['XDIM']/2),
          hdr_dict["ULYMAP"] - hdr_dict['NROWS']*hdr_dict['YDIM'])
    
    lon_point_list = (p1[0], p2[0], p3[0], p4[0])
    lat_point_list = (p1[1], p2[1], p3[1], p4[1])
        
    ppt_bounds = Polygon(zip(lon_point_list, lat_point_list))
    
    return ppt_bounds, ppt_data, hdr_dict

def convert_pptData_to_GDF(ppt_bounds, ppt_data, hdr_dict): 
    Xmin, Ymin, Xmax, Ymax = ppt_bounds.bounds
    
    Xlength = int((Xmax - Xmin)/hdr_dict['XDIM'])
    Ylength = int((Ymax - Ymin)/hdr_dict['YDIM'])
    
    xx, yy = np.meshgrid(np.linspace(Xmin, Xmax, Xlength), np.linspace(Ymin, Ymax, Ylength))
    xc = xx.flatten()
    yc = yy.flatten()
    ppt_data = ppt_data.flatten()
    
    df = pd.DataFrame(
        {'Precipitation': ppt_data,
         'Latitude': yc,
         'Longitude': xc})
        
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

    return gdf
def plot_basins(basin_geoms):
    crs = {'init': 'epsg:4326'}
    m = folium.Map(zoom_start=10, tiles='cartodbpositron')

    for basin_geom in basin_geoms:
        polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[basin_geom])       

        folium.GeoJson(polygon).add_to(m)
        folium.LatLngPopup().add_to(m)
        
    return m

In [68]:
def plot_basins(basin_geoms):
    crs = {'init': 'epsg:4326'}
    m = folium.Map(zoom_start=10, tiles='cartodbpositron')

    for basin_geom in basin_geoms:
        polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[basin_geom])       

        folium.GeoJson(polygon).add_to(m)
        folium.LatLngPopup().add_to(m)
        
    return m

In [ ]:
def convert_basin_geom_to_GDF(basin):
        
    longs, lats = basin.exterior.coords.xy

    df = pd.DataFrame({'Latitude': longs,
                       'Longitude': lats})
        
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
    
    return gdf

In [77]:
def main():
    all_basin_geoms, basin_file_names = get_all_basin_coords()
    ppt_bounds, ppt_data, hdr_dict = get_ppt_data(year=1987, month=8)
    
    ppt_gdf = convert_pptData_to_GDF(ppt_bounds, ppt_data, hdr_dict)
    all_basin_gdf = []
    for basin in all_basin_geoms:
        basin_gdf = convert_basin_geom_to_GDF(basin)
        all_basin_gdf.append(basin_gdf)    
    
    #plot_basins(all_basin_geoms)
    #fig, ax = plt.subplots(1, 1)
    #ppt_gdf.plot(column="Precipitation", ax=ax, legend=True)

    intersected = []
    #clipped = gpd.clip(gdf=ppt_gdf, mask=all_basin_geoms[0])
    spatial_index = ppt_gdf.sindex
    
    for count, basin_geom in enumerate(all_basin_geoms):
        print("Index", count)
        print("basin_file_name:", basin_file_names[count])    
        possible_matches_index = list(spatial_index.intersection(basin_geom.bounds))
        possible_matches = ppt_gdf.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(basin_geom)]
        
        intersected.append(precise_matches)
#    for count, basin_gdf in enumerate(all_basin_gdf):
#        print("Index", count)
#        print("basin_file_name:", basin_file_names[count])
#        z=gpd.overlay(basin_gdf, ppt_gdf, how='intersection')
#        if(z.shape[0]> 0):
#            intersected.append(z)
#            print("found")
#            
    return intersected

In [78]:
inter = main()

Index 0
basin_file_name: 01048000.BDY
Index 1
basin_file_name: 01055500.BDY
Index 2
basin_file_name: 01060000.BDY
Index 3
basin_file_name: 01064500.BDY
Index 4
basin_file_name: 01076500.BDY
Index 5
basin_file_name: 01127000.BDY
Index 6
basin_file_name: 01138000.BDY
Index 7
basin_file_name: 01197500.BDY
Index 8
basin_file_name: 01200000.BDY
Index 9
basin_file_name: 01321000.BDY
Index 10
basin_file_name: 01329000.BDY
Index 11
basin_file_name: 01329500.BDY
Index 12
basin_file_name: 01334500.BDY
Index 13
basin_file_name: 01348000.BDY
Index 14
basin_file_name: 01361000.BDY
Index 15
basin_file_name: 01371500.BDY
Index 16
basin_file_name: 01372500.BDY
Index 17
basin_file_name: 01421000.BDY
Index 18
basin_file_name: 01423000.BDY
Index 19
basin_file_name: 01426500.BDY
Index 20
basin_file_name: 01445500.BDY
Index 21
basin_file_name: 01500500.BDY
Index 22
basin_file_name: 01503000.BDY
Index 23
basin_file_name: 01512500.BDY
Index 24
basin_file_name: 01514000.BDY
Index 25
basin_file_name: 01518000.

Index 209
basin_file_name: 05244000.BDY
Index 210
basin_file_name: 05280000.BDY
Index 211
basin_file_name: 05320500.BDY
Index 212
basin_file_name: 05408000.BDY
Index 213
basin_file_name: 05410490.BDY
Index 214
basin_file_name: 05412500.BDY
Index 215
basin_file_name: 05418500.BDY
Index 216
basin_file_name: 05422000.BDY
Index 217
basin_file_name: 05430500.BDY
Index 218
basin_file_name: 05435500.BDY
Index 219
basin_file_name: 05440000.BDY
Index 220
basin_file_name: 05447500.BDY
Index 221
basin_file_name: 05451500.BDY
Index 222
basin_file_name: 05452000.BDY
Index 223
basin_file_name: 05454500.BDY
Index 224
basin_file_name: 05455500.BDY
Index 225
basin_file_name: 05457700.BDY
Index 226
basin_file_name: 05458500.BDY
Index 227
basin_file_name: 05462000.BDY
Index 228
basin_file_name: 05471500.BDY
Index 229
basin_file_name: 05472500.BDY
Index 230
basin_file_name: 05476500.BDY
Index 231
basin_file_name: 05479000.BDY
Index 232
basin_file_name: 05481000.BDY
Index 233
basin_file_name: 05482500.BDY


In [79]:
inter


[        Precipitation   Latitude  Longitude                    geometry
 701000            NaN  44.841868 -70.379184  POINT (-70.37918 44.84187)
 701002            NaN  44.841868 -70.295762  POINT (-70.29576 44.84187)
 700997            NaN  44.841868 -70.504318  POINT (-70.50432 44.84187)
 702401            NaN  44.883636 -70.546029  POINT (-70.54603 44.88364)
 702405            NaN  44.883636 -70.379184  POINT (-70.37918 44.88364)
 ...               ...        ...        ...                         ...
 692578            NaN  44.591263 -70.045495  POINT (-70.04550 44.59126)
 692577            NaN  44.591263 -70.087206  POINT (-70.08721 44.59126)
 692580            NaN  44.591263 -69.962073  POINT (-69.96207 44.59126)
 692579            NaN  44.591263 -70.003784  POINT (-70.00378 44.59126)
 693978            NaN  44.633031 -70.254051  POINT (-70.25405 44.63303)
 
 [82 rows x 4 columns],
         Precipitation   Latitude  Longitude                    geometry
 685547            NaN  4